In [16]:
## Importing library
import os
import cv2
import numpy as np
from imutils import paths
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Input, AveragePooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import LearningRateScheduler
import pickle

In [ ]:
# Set up paths and parameters
datapath = r"C:\Rajat\COLLEGE\Semester 7\Hyperverge\football_openCV\data"
output_model = r"C:\Rajat\COLLEGE\Semester 7\Hyperverge\football_openCV\model\videoclassificationmodel"
outputlabelbinarize = r"C:\Rajat\COLLEGE\Semester 7\Hyperverge\football_openCV\model\videoclassificationbinarizer"
epoch = 25
Sports_labels = set(['boxing', 'swimming', 'table_tennis'])

In [ ]:
# Load images
print("Loading images...")
pathToImages = list(paths.list_images(datapath))
data = []
labels = []

for images in pathToImages:
    label = images.split(os.path.sep)[-2]
    if label not in Sports_labels:
        continue
    image = cv2.imread(images)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (244, 244))
    data.append(image)
    labels.append(label)

data = np.array(data)
labels = np.array(labels)


In [ ]:
# One-hot encode the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

In [ ]:
# Split the data into training and testing sets
(X_train, X_test, Y_train, Y_test) = train_test_split(data, labels, test_size=0.25, stratify=labels, random_state=42)


In [ ]:
# Data augmentation
trainingAugmentation = ImageDataGenerator(
    rotation_range=30,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest"
)
validationAugmentation = ImageDataGenerator()

mean = np.array([123.68, 116.779, 103.939], dtype="float32")
trainingAugmentation.mean = mean
validationAugmentation.mean = mean


In [ ]:

# Build the model
baseModel = ResNet50(weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 244, 3)))
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(512, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(len(lb.classes_), activation="softmax")(headModel)
model = Model(inputs=baseModel.input, outputs=headModel)

In [ ]:
# Freeze the base layers
for layer in baseModel.layers:
    layer.trainable = False

In [ ]:
# Compile the model with the new optimizer format
opt = SGD(learning_rate=0.0001, momentum=0.9)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

In [ ]:
# Learning rate scheduler
def lr_schedule(epoch, lr):
    decay_rate = 0.0001 / (epoch + 1)
    return lr * decay_rate

lr_scheduler = LearningRateScheduler(lr_schedule)

In [2]:

# Train the model
History = model.fit(
    trainingAugmentation.flow(X_train, Y_train, batch_size=32),
    steps_per_epoch=len(X_train) // 32,
    validation_data=validationAugmentation.flow(X_test, Y_test),
    validation_steps=len(X_test) // 32,
    epochs=epoch,
    callbacks=[lr_scheduler]
)

# Save the model and label binarizer
model.save(output_model)
with open(outputlabelbinarize, "wb") as lbinarizer:
    pickle.dump(lb, lbinarizer)


Loading images...
Epoch 1/25
49/49 [==============================] - 135s 3s/step - loss: 1.6260 - accuracy: 0.3429 - val_loss: 1.2546 - val_accuracy: 0.3594 - lr: 1.0000e-08
Epoch 2/25
49/49 [==============================] - 125s 3s/step - loss: 1.6567 - accuracy: 0.3338 - val_loss: 1.2546 - val_accuracy: 0.3633 - lr: 5.0000e-13
Epoch 3/25
49/49 [==============================] - 129s 3s/step - loss: 1.6909 - accuracy: 0.3364 - val_loss: 1.2462 - val_accuracy: 0.3652 - lr: 1.6667e-17
Epoch 4/25
49/49 [==============================] - 128s 3s/step - loss: 1.6664 - accuracy: 0.3380 - val_loss: 1.2453 - val_accuracy: 0.3672 - lr: 4.1667e-22
Epoch 5/25
49/49 [==============================] - 134s 3s/step - loss: 1.6968 - accuracy: 0.3208 - val_loss: 1.2536 - val_accuracy: 0.3613 - lr: 8.3333e-27
Epoch 6/25
49/49 [==============================] - 127s 3s/step - loss: 1.6538 - accuracy: 0.3507 - val_loss: 1.2508 - val_accuracy: 0.3633 - lr: 1.3889e-31
Epoch 7/25
49/49 [================

INFO:tensorflow:Assets written to: C:\Rajat\COLLEGE\Semester 7\Hyperverge\football_openCV\model\videoclassificationmodel\assets
